In this code block, we load all of the txt files that we imported from Genius's API and create a csv file using the Pandas library

In [33]:
import os
import pandas as pd

# Set the directory where the text files are located
directory = "songs"

# Create an empty DataFrame to store the data
dataframe = pd.DataFrame(columns=["Artist", "Lyrics"])

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        # Read the text file into a string
        with open(os.path.join(directory, filename), "r") as file:
            lyrics = file.read()

        # Get the artist name from the filename
        artist = os.path.splitext(filename)[0]

        # Add the data to the DataFrame
        dataframe = dataframe.append({"Artist": artist, "Lyrics": lyrics}, ignore_index=True)

# Export the DataFrame to a CSV file
dataframe.to_csv("lyrics1.csv", index=False)


In [32]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Load the data from the CSV file
data = pd.read_csv("lyrics1.csv")

nltk.download('stopwords')
nltk.download('punkt')

# Create a list of stopwords to remove
stop_words = set(stopwords.words("english"))
stop_words.add("verse")
# stop_words.add("lyric")

# Create a stemmer to use for word stemming
stemmer = PorterStemmer()

# Preprocess each lyric in the DataFrame
for i, row in data.iterrows():
#Convert the lyric to lowercase
    lyric = str(row["Lyrics"]).lower()
    
    split_lyric = lyric.split("lyric", 1)
    cleaned_lyric = split_lyric[0]
    print(cleaned_lyric)

    #Tokenize the lyric into words
    words = word_tokenize(cleaned_lyric)

    #Remove stop words and punctuation
    filtered_words = [word for word in words if word.isalpha() and word not in stop_words]

    #Stem each word
    stemmed_words = [stemmer.stem(word) for word in filtered_words]

    #Join the stemmed words back into a single string
    preprocessed_lyric = " ".join(stemmed_words)

    #Replace the original lyric with the preprocessed lyric in the DataFrame
    data.at[i, "Lyrics"] = preprocessed_lyric

# Export the preprocessed DataFrame to a CSV file
data.to_csv("preprocessed_lyrics.csv", index=False)




**************************************************
freestyletranslationstürkçefreestyle 

**************************************************
mercytranslationsportuguêsitalianoenglishespañolmercy 
nan

**************************************************
xo tour llif3translationsрусскийportuguêsitalianoxo tour llif3 

**************************************************
god’s plantranslationsрусскийtürkçeespañolpolskiitalianodeutschfrançaisazərbaycanca / آذربايجانgod’s plan 

**************************************************
fuck the world (summer in london)translationsportuguêstürkçefuck the world (summer in london) 

**************************************************
love metranslationsfrançaislove me 

**************************************************
all too well (10 minute version) (taylor’s version) [from the vault]translationsespañolbosanskiрусскийportuguêsnorsk (bokmål / riksmål)françaisفارسیnederlandsall too well (10 minute version) (taylor’s version) [from the vault] 


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jarraomar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jarraomar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(dataframe, test_size=0.2, random_state=42)

# Export the training and testing sets to CSV files
train_data.to_csv("train_lyrics.csv", index=False)
test_data.to_csv("test_lyrics.csv", index=False)
# Export the DataFrame to a CSV file
dataframe.to_csv("lyrics.csv", index=False)